In [34]:
from akashiwo_data_scrape import *
from bs4 import BeautifulSoup
from tqdm import tqdm

In [25]:
# speacies id
spicies_ids = [45, 3]
spicies_names = ["Diaton_info.csv", "shtonela_info.csv"]

for id, names in zip(spicies_ids, spicies_names):
    pull_data(id, "2011/5/01", "2023/10/11", names)

In [26]:
colum_names = ['lat', 'lng', 'speciesValueAM', 'speciesValuePM', 
               'saisuiValueAM', 'saisuiValuePM', 'speciesNameKana', 
               'maxvalue', 'speciesId', 'icon_size']

for name in spicies_names:
    remove_colum(colum_names, name)

In [29]:
def remove_duplicates(list1, list2):
    ziped_list = list(zip(list1, list2))
    return list(OrderedDict.fromkeys(ziped_list))

In [35]:
data = pd.read_csv("shtonela_info.csv")
pointIds = data["pointId"]
gatherYMDs = data["gatherYMD"]
filtered_dictionary = remove_duplicates(pointIds, gatherYMDs)

In [ ]:
main_data = {}
coordinate_data = {}
headers_to_skip_main = ["確定値／速報値", "事業・調査名"]
i = 0
for pointId, gatherYMD in filtered_dictionary:
    html_data = requests.get(f"https://akashiwo.jp/private/akashiwoListInit.php?qpoint_id={str(pointId)}&qspecies_id=3&qgather_ymd_s=&qgather_ymd_e={str(gatherYMD)}")
    html_data.encoding = 'utf-8'
    soup = BeautifulSoup(html_data.text, 'html.parser')
    tables = soup.find_all('table')
    times_to_duplicate = parse_main_table(tables[1], headers_to_skip_main, main_data)
    parse_coordinate_table(tables[0], coordinate_data, times_to_duplicate)
    if(i == 20): break
    i += 1

df = pd.DataFrame(main_data)
df.to_csv("hehe.csv", index=False)